Importing relevant python and Hugging Face packages and defining the tokenizer:

In [1]:
import torch
import pandas as pd
from google.colab import drive
drive.mount('/gdrive')
!pip install transformers
from transformers import BertTokenizer, BertModel
from scipy.spatial.distance import cosine
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Mounted at /gdrive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 76.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 90.4 MB/s eta 0:00:00


Importing and formatting CSV data for each corpus:

In [ ]:
cofeaNonLegal = pd.read_csv("/gdrive/My Drive/Colab Notebooks/cofeaNonLegal.csv") 
right = cofeaNonLegal.iloc[:,2]
free = cofeaNonLegal.iloc[:,1]
cofeaNonLegal["text"] = cofeaNonLegal["Left"].str.cat(free)
cofeaNonLegal["text"] = cofeaNonLegal["text"].str.cat(right)

In [ ]:
cofeaLegal = pd.read_csv("/gdrive/My Drive/Colab Notebooks/cofeaLegal.csv") 
right = cofeaNonLegal.iloc[:,2]
free = cofeaNonLegal.iloc[:,1]
cofeaLegal["text"] = cofeaLegal["Left"].str.cat(free)
cofeaLegal["text"] = cofeaLegal["text"].str.cat(right)

In [2]:
cofea = pd.read_csv("/gdrive/My Drive/Colab Notebooks/cofea.csv") 
right = cofea.iloc[:,2]
free = cofea.iloc[:,1]
cofea["text"] = cofea["Left"].str.cat(free)
cofea["text"] = cofea["text"].str.cat(right)

In [2]:
cuscoNewData = pd.read_csv("/gdrive/My Drive/Colab Notebooks/coscousMod.csv") 
right = cuscoNewData.iloc[:,2]
free = cuscoNewData.iloc[:,1]
cuscoNewData["text"] = cuscoNewData["Left"].str.cat(free)
cuscoNewData["text"] = cuscoNewData["text"].str.cat(right)

In [4]:
nowData = pd.read_csv("/gdrive/My Drive/Colab Notebooks/now.csv")
nowData["text"] = nowData.iloc[:,2]

Indexing keywords:

In [1]:
indexed = []
#indices for our keywords:
freedomIndex = tokenizer.get_vocab().get('freedom', tokenizer.unk_token_id)
fromIndex = tokenizer.get_vocab().get('from', tokenizer.unk_token_id)
toIndex = tokenizer.get_vocab().get('to', tokenizer.unk_token_id)
ofIndex = tokenizer.get_vocab().get('of', tokenizer.unk_token_id)
libertyIndex = tokenizer.get_vocab().get('liberty', tokenizer.unk_token_id)


#for bounds
#expecting little change:
eveningIndex = tokenizer.get_vocab().get('evening', tokenizer.unk_token_id)
salaryIndex = tokenizer.get_vocab().get('salary', tokenizer.unk_token_id)
firstIndex = tokenizer.get_vocab().get('first', tokenizer.unk_token_id)

#expecting more change:
nationalityIndex = tokenizer.get_vocab().get('nationality', tokenizer.unk_token_id)
religiousIndex = tokenizer.get_vocab().get('religious', tokenizer.unk_token_id)
awfulIndex = tokenizer.get_vocab().get('awful', tokenizer.unk_token_id)
specificIndeces = {"freedom": freedomIndex,
                   "from": fromIndex,
                   "to": toIndex,
                   "of": ofIndex,
                   "liberty": libertyIndex, 
                   "evening": eveningIndex, 
                   "salary": salaryIndex, 
                   "first": firstIndex,
                   "nationality": nationalityIndex, 
                   "religious": religiousIndex, 
                   "awful": awfulIndex}

NameError: ignored

In [4]:
def index(data):
  tokenizedSentences = []
  indexedSentences = []
  sentenceID = []
  i = 0

  for sentence in data["text"]:
    tokenized = tokenizer.tokenize(str(sentence))
    tokenizedSentences.append(tokenized)
    indexedSentences.append(tokenizer.convert_tokens_to_ids(tokenized))
    sentenceID.append([i] * len(tokenized))
    i += 1

  indexedTensor = []
  IDTensor = []
  for sentence in indexedSentences:
    indexedTensor.append(torch.tensor(sentence))

  for x in sentenceID:
    IDTensor.append(torch.tensor(x))

  return indexedSentences, indexedTensor, IDTensor

A function for getting hidden states:

In [5]:
def getStates(model,indexedTensor, IDTensor):
  outputs = []
  states = []
  i = 0
  #get indexing
  for sentence in indexedTensor:
    with torch.no_grad():
      output = model(sentence.unsqueeze(0), IDTensor[i].unsqueeze(0))
      states.append(output[2])
      outputs.append(output) 
      i += 1

  return outputs, states

Summing the final four states to get a vector:

In [6]:
def getSumVectors(embeddings):
  sumVec = []
  for vec in embeddings:
    vectors = []
    for token in vec:
      vectors.append(torch.nansum(token[:4], dim=0))
    sumVec.append(vectors)
  return sumVec

Categorize by the word that follows "freedom":

In [7]:
def categorize(indexedSentences, sumVec):
  freedomsFrom = []
  freedomsTo = []
  freedomsOf = []
  freedoms = []
  for i in range(len(sumVec)):
    for w in range(len(indexedSentences[i]) - 1):
      if indexedSentences[i][w] == freedomIndex:
        freedoms.append(sumVec[i][w])
        if indexedSentences[i][w + 1] == fromIndex:
          freedomsFrom.append(sumVec[i][w])
        elif indexedSentences[i][w + 1] == toIndex:
          freedomsTo.append(sumVec[i][w])
        elif indexedSentences[i][w + 1] == ofIndex:
          freedomsOf.append(sumVec[i][w])
  return [freedomsFrom, freedomsTo, freedomsOf, freedoms]

Get embeddings for a specific (non-"freedom") word:

In [8]:
def specific(indexedSentences, sumVec, word):
  wordIndex = specificIndeces[word]
  embeddings = []
  for i in range(len(sumVec)):
    for w in range(len(indexedSentences[i]) - 1):
     if indexedSentences[i][w] == wordIndex:
        embeddings.append(sumVec[i][w])
  return embeddings

Get embeddings for a specific word in a corpus:

In [9]:
def embed(data, word):
  indexedSentences, indexedTensor, IDTensor = index(data)

  #set up model
  model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states = True)
  model.eval()
  outputs, states = getStates(model, indexedTensor, IDTensor)

  #getting embeddings
  embeddings = []
  for state in states:
    x = torch.stack(state, dim=0)
    y = torch.squeeze(x, dim=1)
    z = y.permute(1,0,2)
    embeddings.append(z)
  sumVec = getSumVectors(embeddings)
  #if word == "freedom":
    #return categorize(indexedSentences, sumVec)
  #else:
  return specific(indexedSentences, sumVec, word)


Below are a selection of modeled embeddings for "freedom" in the relevant corpora:

In [ ]:
cofeaNL = embed(cofeaNonLegal, "freedom")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
cofeaL = embed(cofeaLegal, "freedom")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [16]:
cuscoNew = embed(cuscoNewData, "freedom")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [17]:
now = embed(nowData, "freedom")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [22]:
cuscoLiberties = embed(cuscoNewData, "liberty")
nowLiberties = embed(nowData, "liberty")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.La

In [25]:
cuscoLiberty = torch.mean(torch.stack(cuscoLiberties), axis = 0)
nowLiberty = torch.mean(torch.stack(nowLiberties), axis = 0)
cuscoFreedom = torch.mean(torch.stack(cuscoNew), axis = 0)
nowFreedom = torch.mean(torch.stack(now), axis = 0)
print("Similarity between liberty and freedom for CUSCO: ", 1-cosine(cuscoFreedom,cuscoLiberty))
print("Similarity between liberty and freedom for NOW: ", 1-cosine(nowFreedom, nowLiberty))

coscous:  0.47172674536705017
now:  0.4684829115867615


In [ ]:
cofeaNLfroms, cofeaLfroms, cuscoNewfroms= cofeaNL[0], cofeaL[0], cuscoNew[0]
cofeaNLtos, cofeaLtos, cuscoNewtos = cofeaNL[1], cofeaL[1], cuscoNew[1]
cofeaNLofs, cofeaLofs, cuscoNewofs = cofeaNL[2], cofeaL[2], cuscoNew[2]
cofeaNLall, cofeaLall, cuscoNewall = cofeaNL[3], cofeaL[3], cuscoNew[3]

Calculating means for comparison:

In [ ]:
cofeaNL = torch.mean(torch.stack(cofeaNLall), axis = 0)
cofeaL = torch.mean(torch.stack(cofeaLall), axis = 0)
cofeaNLof = torch.mean(torch.stack(cofeaNLofs), axis = 0)
cofeaLof = torch.mean(torch.stack(cofeaLofs), axis = 0)
cofeaNLfrom = torch.mean(torch.stack(cofeaNLfroms), axis = 0)
cofeaLfrom = torch.mean(torch.stack(cofeaLfroms), axis = 0)
cofeaNLto = torch.mean(torch.stack(cofeaNLtos), axis = 0)
cofeaLto = torch.mean(torch.stack(cofeaLtos), axis = 0)
cuscoNewfrom = torch.mean(torch.stack(cuscoNewfroms), axis = 0)
cuscoNewto = torch.mean(torch.stack(cuscoNewtos), axis = 0)
cuscoNewof = torch.mean (torch.stack(cuscoNewofs), axis = 0)
cuscoNew = torch.mean(torch.stack(cuscoNewall), axis = 0)
nowAll = torch.mean(torch.stack(now[3]), axis = 0)
nowOf = torch.mean(torch.stack(now[2]), axis = 0)
nowTo = torch.mean(torch.stack(now[1]), axis = 0)
nowFrom = torch.mean(torch.stack(now[0]), axis = 0)

An example of how we can compare the means:

In [2]:
print("Comparing freedom from and freedom to in earlier and latter legal corpora:", 1-cosine(cofeaLfrom,cuscoNewto))
print("Comparing freedom from and freedom to in earlier and latter non-legal corpora:", 1-cosine(cofeaNLfrom, nowTo))

NameError: ignored